# Kalshi and Polymarket API Test
### Kalshi and Polymarket Individual Tests
- Connect to the API
- See balance and active positions
- Look at active market data
- Place limit order (not executed)
### Joint Test
- Find and compare same events on Kalshi and Polymarket
- Discover potential arbitrage opportunities

#### 0 - Imports

In [1]:
import requests
import datetime
import base64
from pathlib import Path
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.backends import default_backend

# Polymarket SDK
from py_clob_client.client import ClobClient
from py_clob_client.clob_types import OrderArgs, OrderType
from py_clob_client.order_builder.constants import BUY, SELL

import importlib
import config
importlib.reload(config)

from config import (
    KALSHI_API_KEY, KALSHI_PRIVATE_KEY_PATH, KALSHI_BASE_URL,
    POLYMARKET_PRIVATE_KEY, POLYMARKET_PROXY_ADDRESS, POLYMARKET_BASE_URL,
    POLYMARKET_SIGNATURE_TYPE, POLYMARKET_CHAIN_ID, validate_config
)

## 1 - Kalshi

#### 1.1 - Connect to Kalshi

In [2]:
validate_config()

# Load private key
with open(Path(KALSHI_PRIVATE_KEY_PATH), 'rb') as f:
    kalshi_private_key = serialization.load_pem_private_key(
        f.read(), password=None, backend=default_backend()
    )

def connect_kalshi():
    """Initialize Kalshi connection and return authenticated request function."""
    def kalshi_get(path):
        """Make authenticated GET request to Kalshi API."""
        timestamp = str(int(datetime.datetime.now().timestamp() * 1000))
        full_path = f"/trade-api/v2{path}"
        
        message = f"{timestamp}GET{full_path}".encode('utf-8')
        signature = kalshi_private_key.sign(
            message,
            padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.DIGEST_LENGTH),
            hashes.SHA256()
        )
        
        headers = {
            'KALSHI-ACCESS-KEY': KALSHI_API_KEY,
            'KALSHI-ACCESS-SIGNATURE': base64.b64encode(signature).decode('utf-8'),
            'KALSHI-ACCESS-TIMESTAMP': timestamp
        }
        
        return requests.get(KALSHI_BASE_URL + path, headers=headers)
    return kalshi_get

# Test connection
kalshi_get = connect_kalshi()
response = kalshi_get("/portfolio/balance")
status = "SUCCESS" if response.status_code == 200 else "FAILED"
print(f"[{status}] Kalshi status: {response.status_code}")

[SUCCESS] Kalshi status: 200


#### 1.2 List Holding Details for Kalshi

In [3]:
def get_kalshi_holdings():
    """Get Kalshi account balance and positions."""
    balance = kalshi_get("/portfolio/balance").json()
    print("=== Balance ===")
    print(f"Available: ${balance['balance'] / 100:.2f}")
    print()
    
    positions = kalshi_get("/portfolio/positions?settlement_status=unsettled").json()
    print("=== Positions ===")
    if positions.get('market_positions'):
        print(f"Active positions: {len(positions['market_positions'])}\n")
        for i, pos in enumerate(positions['market_positions'][:5], 1):
            print(f"{i}. {pos['ticker']} - Position: {pos['position']}, Traded: {pos['total_traded']}")
    else:
        print("No active positions")

get_kalshi_holdings()

=== Balance ===
Available: $9.50

=== Positions ===
No active positions


#### 1.3 Kalshi Market Data

#### 1.4 Kalshi Place Limit Order (Not Executed)

## 2 - Polymarket

#### 2.1 - Connect to Polymarket

In [4]:
def connect_polymarket():
    """Initialize Polymarket client using py-clob-client SDK."""
    if not POLYMARKET_PRIVATE_KEY:
        print("WARNING: POLYMARKET_PRIVATE_KEY not set in .env")
        return None
    if not POLYMARKET_PROXY_ADDRESS:
        print("WARNING: POLYMARKET_PROXY_ADDRESS not set in .env")
        return None
    
    try:
        client = ClobClient(
            POLYMARKET_BASE_URL,
            key=POLYMARKET_PRIVATE_KEY,
            chain_id=POLYMARKET_CHAIN_ID,
            signature_type=POLYMARKET_SIGNATURE_TYPE,
            funder=POLYMARKET_PROXY_ADDRESS
        )
        client.set_api_creds(client.create_or_derive_api_creds())
        return client
    except Exception as e:
        print(f"[FAILED] Failed to initialize: {e}")
        return None

# Test connection
polymarket_client = connect_polymarket()
status = "SUCCESS" if polymarket_client else "FAILED"
print(f"[{status}] Polymarket status: {200 if polymarket_client else 0}")

[SUCCESS] Polymarket status: 200


#### 2.2 List Holding Details for Polymarket

In [5]:
def get_polymarket_holdings():
    """Get Polymarket account balance and positions."""
    from py_clob_client.clob_types import BalanceAllowanceParams, AssetType, OpenOrderParams
    
    balance = polymarket_client.get_balance_allowance(
        params=BalanceAllowanceParams(asset_type=AssetType.COLLATERAL)
    )
    print("=== Balance ===")
    print(f"Available: ${float(balance.get('balance', 0)):.2f}")
    print()
    
    orders = polymarket_client.get_orders(OpenOrderParams())
    print("=== Positions ===")
    if orders:
        print(f"Active positions: {len(orders)}\n")
        for i, order in enumerate(orders[:5], 1):
            print(f"{i}. {order.get('market', 'N/A')} - Side: {order.get('side', 'N/A')}, Size: {order.get('size', 0)}")
    else:
        print("No active positions")

get_polymarket_holdings()

=== Balance ===
Available: $0.00

=== Positions ===
No active positions


#### 2.3 Polymarket Market Data

In [8]:
def get_polymarket_markets(limit=5):
    """Get active Polymarket markets using Gamma API."""
    import json
    
    gamma_url = "https://gamma-api.polymarket.com/events"
    params = {
        'limit': limit,
        'closed': 'false',
        'order': 'id',
        'ascending': 'false'
    }
    
    response = requests.get(gamma_url, params=params)
    if response.status_code != 200:
        print(f"ERROR: Failed to fetch markets ({response.status_code})")
        return None
    
    events = response.json()
    
    print(f"=== Markets (Top {limit}) ===\n")
    for i, event in enumerate(events[:limit], 1):
        print(f"{i}. {event.get('title', 'N/A')}")
        print(f"   Slug: {event.get('slug', 'N/A')}")
        
        # End date (when the event resolves/contract finishes)
        end_date = event.get('endDate', 'N/A')
        if end_date != 'N/A':
            print(f"   Resolves: {end_date}")
        
        # Volume
        volume = event.get('volume', 0)
        if volume:
            print(f"   Volume: ${float(volume):,.2f}")
        
        # Markets within event
        markets = event.get('markets', [])
        if markets:
            for market in markets:
                # Parse outcomes (JSON string)
                outcomes_str = market.get('outcomes', '[]')
                outcomes = json.loads(outcomes_str) if isinstance(outcomes_str, str) else outcomes_str
                
                # Parse prices (JSON string)
                prices_str = market.get('outcomePrices', '[]')
                prices = json.loads(prices_str) if isinstance(prices_str, str) else prices_str
                
                # Display each outcome with its price
                for j, outcome in enumerate(outcomes):
                    price = prices[j] if j < len(prices) else 'N/A'
                    if price != 'N/A':
                        print(f"   {outcome}: ${float(price):.4f}")
                    else:
                        print(f"   {outcome}: N/A")
        print()
    
    return events

markets = get_polymarket_markets()

=== Markets (Top 5) ===

1. Ethereum Up or Down - December 21, 2:55AM-3:00AM ET
   Slug: eth-updown-5m-1766303700
   Resolves: 2025-12-21T08:00:00Z
   Up: N/A
   Down: N/A

2. XRP Up or Down - December 21, 2:55AM-3:00AM ET
   Slug: xrp-updown-5m-1766303700
   Resolves: 2025-12-21T08:00:00Z
   Up: N/A
   Down: N/A

3. Solana Up or Down - December 21, 2:55AM-3:00AM ET
   Slug: sol-updown-5m-1766303700
   Resolves: 2025-12-21T08:00:00Z
   Up: N/A
   Down: N/A

4. Bitcoin Up or Down - December 21, 2:55AM-3:00AM ET
   Slug: btc-updown-5m-1766303700
   Resolves: 2025-12-21T08:00:00Z
   Up: N/A
   Down: N/A

5. Ethereum Up or Down - December 21, 3:00AM-3:15AM ET
   Slug: eth-updown-15m-1766304000
   Resolves: 2025-12-21T08:15:00Z
   Up: N/A
   Down: N/A



#### 2.4 Polymarket Place Limit Order (Not Executed)

In [7]:
def place_limit_order_example(token_id, price, size, side=BUY, execute=False):
    """
    Function to place a limit order on Polymarket.
    
    Args:
        token_id: Token ID to trade
        price: Price per token (0.01 to 0.99)
        size: Number of tokens to buy/sell
        side: BUY or SELL
        execute: If False, only shows what would be done (default: False)
    """
    if not polymarket_client:
        print("WARNING: Polymarket client not initialized")
        return None
    
    print(f"=== {'EXECUTING' if execute else 'EXAMPLE'} Limit Order ===")
    print(f"Side: {side}")
    print(f"Token ID: {token_id}")
    print(f"Price: ${price:.4f}")
    print(f"Size: {size} tokens")
    print(f"Total Cost: ${price * size:.2f} USDC")
    print()
    
    if not execute:
        print("WARNING: Order NOT executed (execute=False)")
        print("Set execute=True to place real order")
        return None
    
    try:
        # Create order arguments
        order_args = OrderArgs(
            price=price,
            size=size,
            side=side,
            token_id=token_id
        )
        
        # Sign the order
        signed_order = polymarket_client.create_order(order_args)
        
        # Post order as GTC (Good-Till-Cancelled)
        response = polymarket_client.post_order(signed_order, OrderType.GTC)
        
        print("[SUCCESS] Order placed successfully!")
        print(f"Order ID: {response.get('orderID', 'N/A')}")
        print(f"Status: {response.get('status', 'N/A')}")
        
        return response
        
    except Exception as e:
        print(f"ERROR: Error placing order: {e}")
        return None

# Example: Buy 5 tokens at $0.01 each (NOT executed)
example_order = place_limit_order_example(
    token_id="21742633143463906290569050155826241533067272736897614950488156847949938836455",
    price=0.01,
    size=5.0,
    side=BUY,
    execute=False  # Set to True to actually place the order
)

=== EXAMPLE Limit Order ===
Side: BUY
Token ID: 21742633143463906290569050155826241533067272736897614950488156847949938836455
Price: $0.0100
Size: 5.0 tokens
Total Cost: $0.05 USDC

Set execute=True to place real order


## 3 - Joint Tests